# Notebook 03: Multiprocessing & CPU-Bound Tasks

## Topic 3: True Parallelism for Heavy Computation

In this notebook, we'll explore:
1. Why threads fail for CPU-bound tasks
2. `ProcessPoolExecutor` for true parallelism
3. Monte Carlo option pricing in parallel
4. Chunking to reduce overhead
5. Portfolio Value-at-Risk (VaR) calculation

---

In [ ]:
import numpy as np
import pandas as pd
import time
import os
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import matplotlib.pyplot as plt

n_cores = os.cpu_count()
print(f"This machine has {n_cores} CPU cores")

---

## Section 1: Why Threads Fail for CPU-Bound Tasks

Let's see the GIL in action. We'll run a CPU-intensive task and compare threads vs processes.

In [ ]:
def cpu_intensive_task(n):
    """
    A CPU-intensive task: compute sum of squares.
    This keeps the CPU busy with calculations.
    """
    total = 0.0
    for i in range(n):
        total += i * i * np.sin(i) * np.cos(i)
    return total

# Test the function
start = time.time()
result = cpu_intensive_task(1_000_000)
print(f"Single task time: {time.time() - start:.2f}s")

In [ ]:
# Run multiple CPU-intensive tasks
n_tasks = 4
task_size = 2_000_000

print(f"Running {n_tasks} CPU-intensive tasks...\n")

# Sequential
start = time.time()
sequential_results = [cpu_intensive_task(task_size) for _ in range(n_tasks)]
seq_time = time.time() - start
print(f"Sequential:     {seq_time:.2f}s")

# Threading (will NOT help due to GIL)
start = time.time()
with ThreadPoolExecutor(max_workers=n_tasks) as executor:
    thread_results = list(executor.map(lambda _: cpu_intensive_task(task_size), range(n_tasks)))
thread_time = time.time() - start
print(f"Threading:      {thread_time:.2f}s (speedup: {seq_time/thread_time:.2f}x)")

# Multiprocessing (WILL help - bypasses GIL)
start = time.time()
with ProcessPoolExecutor(max_workers=n_tasks) as executor:
    process_results = list(executor.map(cpu_intensive_task, [task_size] * n_tasks))
process_time = time.time() - start
print(f"Multiprocessing: {process_time:.2f}s (speedup: {seq_time/process_time:.2f}x)")

### Key Observation

- **Threading**: No speedup (or even slower!) because the GIL prevents true parallel execution
- **Multiprocessing**: Near-linear speedup because each process has its own Python interpreter and GIL

**Rule**: For CPU-bound work, always use `ProcessPoolExecutor`!

---

## Section 2: Monte Carlo Option Pricing

Monte Carlo simulation is the bread and butter of quantitative finance. Let's parallelize it!

### Black-Scholes-Merton Model

Under risk-neutral pricing, the stock price at maturity is:

$$S_T = S_0 \exp\left[\left(r - \frac{\sigma^2}{2}\right)T + \sigma\sqrt{T}Z\right]$$

Where $Z \sim N(0,1)$

In [ ]:
def monte_carlo_option_batch(args):
    """
    Price a European call option using Monte Carlo simulation.
    
    This function is designed for parallel execution - it takes a single
    tuple of arguments and returns the average payoff for a batch of simulations.
    """
    S0, K, T, r, sigma, n_paths, seed = args
    
    # Set seed for reproducibility
    np.random.seed(seed)
    
    # Generate random paths
    Z = np.random.standard_normal(n_paths)
    
    # Simulate terminal stock prices
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    
    # Calculate payoffs
    payoffs = np.maximum(ST - K, 0)
    
    # Return discounted average payoff
    return np.exp(-r * T) * np.mean(payoffs)

In [ ]:
# Option parameters
S0 = 100      # Current stock price
K = 100       # Strike price (at-the-money)
T = 1.0       # 1 year to maturity
r = 0.05      # 5% risk-free rate
sigma = 0.2   # 20% volatility

# Simulation parameters
total_paths = 4_000_000  # Total number of Monte Carlo paths
n_batches = 8            # Split into batches for parallel processing
paths_per_batch = total_paths // n_batches

print(f"Total paths: {total_paths:,}")
print(f"Batches: {n_batches}")
print(f"Paths per batch: {paths_per_batch:,}")

In [ ]:
# Prepare arguments for each batch
batch_args = [
    (S0, K, T, r, sigma, paths_per_batch, seed) 
    for seed in range(n_batches)
]

# Sequential execution
print("Sequential execution:")
start = time.time()
sequential_prices = [monte_carlo_option_batch(args) for args in batch_args]
seq_time = time.time() - start
seq_price = np.mean(sequential_prices)
print(f"  Time: {seq_time:.2f}s")
print(f"  Option Price: ${seq_price:.4f}")

In [ ]:
# Parallel execution
print("\nParallel execution:")
start = time.time()
with ProcessPoolExecutor(max_workers=n_cores) as executor:
    parallel_prices = list(executor.map(monte_carlo_option_batch, batch_args))
par_time = time.time() - start
par_price = np.mean(parallel_prices)
print(f"  Time: {par_time:.2f}s")
print(f"  Option Price: ${par_price:.4f}")
print(f"  Speedup: {seq_time/par_time:.2f}x")

In [ ]:
# Compare with Black-Scholes analytical solution
from scipy.stats import norm

def black_scholes_call(S0, K, T, r, sigma):
    """Analytical Black-Scholes price for European call option."""
    d1 = (np.log(S0/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

bs_price = black_scholes_call(S0, K, T, r, sigma)
print(f"\nBlack-Scholes analytical price: ${bs_price:.4f}")
print(f"Monte Carlo estimate: ${par_price:.4f}")
print(f"Error: ${abs(par_price - bs_price):.4f} ({abs(par_price - bs_price)/bs_price*100:.2f}%)")

---

## Section 3: When Parallelization Hurts - The Overhead Trap

Parallelization has overhead (creating processes, copying data). If tasks are too small, the overhead dominates.

In [ ]:
def tiny_task(x):
    """A very small task - just multiply."""
    return x * x

def small_task(x):
    """A small task - some computation."""
    return sum(i * i for i in range(1000))

def medium_task(x):
    """A medium task - more computation."""
    return sum(i * i for i in range(100_000))

def large_task(x):
    """A large task - substantial computation."""
    return sum(i * i for i in range(1_000_000))

In [ ]:
n_items = 100
items = list(range(n_items))

results = []

for task_name, task_func in [('tiny', tiny_task), ('small', small_task), 
                              ('medium', medium_task), ('large', large_task)]:
    # Sequential
    start = time.time()
    _ = [task_func(x) for x in items]
    seq_time = time.time() - start
    
    # Parallel
    start = time.time()
    with ProcessPoolExecutor(max_workers=4) as executor:
        _ = list(executor.map(task_func, items))
    par_time = time.time() - start
    
    speedup = seq_time / par_time
    results.append({
        'Task': task_name,
        'Sequential (s)': seq_time,
        'Parallel (s)': par_time,
        'Speedup': speedup,
        'Verdict': 'Use parallel' if speedup > 1.5 else 'Stay sequential'
    })
    
results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

### The Lesson

- **Tiny tasks**: Parallel is SLOWER (overhead dominates)
- **Small tasks**: Might be slower or break even
- **Medium/Large tasks**: Parallel wins!

**Rule of thumb**: Each task should take at least 10-100ms to justify parallelization overhead.

### Solution: Chunking

If you have many small tasks, group them into larger chunks.

In [ ]:
def process_chunk(chunk):
    """Process a chunk of items together."""
    return [small_task(x) for x in chunk]

# Create chunks
n_items = 1000
items = list(range(n_items))
chunk_size = 250
chunks = [items[i:i+chunk_size] for i in range(0, n_items, chunk_size)]

print(f"Total items: {n_items}")
print(f"Chunk size: {chunk_size}")
print(f"Number of chunks: {len(chunks)}")

In [ ]:
# Sequential (item by item)
start = time.time()
_ = [small_task(x) for x in items]
seq_time = time.time() - start
print(f"Sequential: {seq_time:.3f}s")

# Parallel (item by item) - bad!
start = time.time()
with ProcessPoolExecutor(max_workers=4) as executor:
    _ = list(executor.map(small_task, items))
par_bad_time = time.time() - start
print(f"Parallel (item by item): {par_bad_time:.3f}s (speedup: {seq_time/par_bad_time:.2f}x)")

# Parallel (chunked) - good!
start = time.time()
with ProcessPoolExecutor(max_workers=4) as executor:
    chunk_results = list(executor.map(process_chunk, chunks))
# Flatten results
_ = [item for chunk in chunk_results for item in chunk]
par_good_time = time.time() - start
print(f"Parallel (chunked): {par_good_time:.3f}s (speedup: {seq_time/par_good_time:.2f}x)")

---

## Section 4: Portfolio Value-at-Risk (VaR)

VaR is a fundamental risk measure in finance. Monte Carlo VaR is computationally intensive and perfect for parallelization.

**VaR**: The maximum expected loss at a given confidence level over a specific time horizon.

In [ ]:
# Create a sample portfolio
np.random.seed(42)

# Portfolio: 5 stocks with different expected returns and volatilities
assets = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']
weights = np.array([0.25, 0.20, 0.25, 0.15, 0.15])  # Portfolio weights
expected_returns = np.array([0.12, 0.10, 0.11, 0.15, 0.20])  # Annual returns
volatilities = np.array([0.25, 0.22, 0.20, 0.30, 0.50])  # Annual volatility

# Correlation matrix (simplified)
correlation = np.array([
    [1.0, 0.6, 0.7, 0.5, 0.3],
    [0.6, 1.0, 0.8, 0.6, 0.4],
    [0.7, 0.8, 1.0, 0.5, 0.3],
    [0.5, 0.6, 0.5, 1.0, 0.5],
    [0.3, 0.4, 0.3, 0.5, 1.0]
])

# Covariance matrix
cov_matrix = np.outer(volatilities, volatilities) * correlation

portfolio_value = 1_000_000  # $1 million portfolio
time_horizon = 10  # 10 trading days
confidence_level = 0.95  # 95% VaR

print("Portfolio:")
for asset, weight in zip(assets, weights):
    print(f"  {asset}: {weight*100:.0f}%")
print(f"\nTotal Value: ${portfolio_value:,}")
print(f"Time Horizon: {time_horizon} days")
print(f"Confidence Level: {confidence_level*100}%")

In [ ]:
def simulate_portfolio_returns(args):
    """
    Simulate portfolio returns over the time horizon.
    Returns the portfolio values at the end of the horizon.
    """
    weights, expected_returns, cov_matrix, n_sims, time_horizon, seed = args
    
    np.random.seed(seed)
    n_assets = len(weights)
    
    # Daily parameters (assuming 252 trading days per year)
    daily_returns = expected_returns / 252
    daily_cov = cov_matrix / 252
    
    # Cholesky decomposition for correlated random numbers
    L = np.linalg.cholesky(daily_cov)
    
    # Simulate returns for each day and each simulation
    portfolio_returns = np.zeros(n_sims)
    
    for sim in range(n_sims):
        cumulative_return = 1.0
        for day in range(time_horizon):
            # Generate correlated random returns
            Z = np.random.standard_normal(n_assets)
            asset_returns = daily_returns + L @ Z
            
            # Portfolio return for this day
            portfolio_daily_return = np.dot(weights, asset_returns)
            cumulative_return *= (1 + portfolio_daily_return)
        
        portfolio_returns[sim] = cumulative_return - 1  # Total return over horizon
    
    return portfolio_returns

In [ ]:
# Simulation parameters
total_simulations = 100_000
n_batches = 8
sims_per_batch = total_simulations // n_batches

print(f"Total simulations: {total_simulations:,}")
print(f"Batches: {n_batches}")
print(f"Simulations per batch: {sims_per_batch:,}")

In [ ]:
# Prepare batch arguments
batch_args = [
    (weights, expected_returns, cov_matrix, sims_per_batch, time_horizon, seed)
    for seed in range(n_batches)
]

# Sequential execution
print("Sequential VaR calculation:")
start = time.time()
seq_returns = [simulate_portfolio_returns(args) for args in batch_args]
seq_returns = np.concatenate(seq_returns)
seq_time = time.time() - start
print(f"  Time: {seq_time:.2f}s")

In [ ]:
# Parallel execution
print("\nParallel VaR calculation:")
start = time.time()
with ProcessPoolExecutor(max_workers=n_cores) as executor:
    par_returns = list(executor.map(simulate_portfolio_returns, batch_args))
par_returns = np.concatenate(par_returns)
par_time = time.time() - start
print(f"  Time: {par_time:.2f}s")
print(f"  Speedup: {seq_time/par_time:.2f}x")

In [ ]:
# Calculate VaR
var_95 = np.percentile(par_returns, (1 - confidence_level) * 100) * portfolio_value
var_99 = np.percentile(par_returns, 1) * portfolio_value
cvar_95 = par_returns[par_returns <= np.percentile(par_returns, 5)].mean() * portfolio_value

print(f"\n{'='*50}")
print(f"RISK METRICS (10-day horizon)")
print(f"{'='*50}")
print(f"95% VaR:  ${-var_95:,.0f} ({-var_95/portfolio_value*100:.2f}%)")
print(f"99% VaR:  ${-var_99:,.0f} ({-var_99/portfolio_value*100:.2f}%)")
print(f"95% CVaR: ${-cvar_95:,.0f} ({-cvar_95/portfolio_value*100:.2f}%)")
print(f"\nInterpretation: With 95% confidence, the portfolio will not")
print(f"lose more than ${-var_95:,.0f} over the next 10 trading days.")

In [ ]:
# Visualize the return distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of returns
ax1 = axes[0]
ax1.hist(par_returns * 100, bins=100, density=True, alpha=0.7, color='steelblue', edgecolor='white')
ax1.axvline(var_95/portfolio_value * 100, color='red', linestyle='--', linewidth=2, label=f'95% VaR: {var_95/portfolio_value*100:.2f}%')
ax1.axvline(var_99/portfolio_value * 100, color='darkred', linestyle='--', linewidth=2, label=f'99% VaR: {var_99/portfolio_value*100:.2f}%')
ax1.axvline(0, color='black', linestyle='-', linewidth=1)
ax1.set_xlabel('Portfolio Return (%)')
ax1.set_ylabel('Density')
ax1.set_title('Distribution of 10-Day Portfolio Returns')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Left tail (losses)
ax2 = axes[1]
losses = par_returns[par_returns < 0] * portfolio_value
ax2.hist(losses, bins=50, density=True, alpha=0.7, color='coral', edgecolor='white')
ax2.axvline(var_95, color='red', linestyle='--', linewidth=2, label=f'95% VaR: ${-var_95:,.0f}')
ax2.axvline(var_99, color='darkred', linestyle='--', linewidth=2, label=f'99% VaR: ${-var_99:,.0f}')
ax2.set_xlabel('Portfolio Loss ($)')
ax2.set_ylabel('Density')
ax2.set_title('Distribution of Losses (Left Tail)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Summary: Multiprocessing for CPU-Bound Tasks

### Key Takeaways

1. **Use `ProcessPoolExecutor` for CPU-bound work** - bypasses the GIL

2. **Threading fails for CPU tasks** - the GIL prevents parallel execution

3. **Watch out for overhead** - tasks should be substantial (>10ms)

4. **Chunking helps** - group small tasks into larger batches

5. **Functions must be picklable** - no lambdas, no nested functions

### Finance Applications

| Application | Why It's CPU-Bound |
|-------------|--------------------|
| Monte Carlo pricing | Millions of path simulations |
| VaR calculations | Many portfolio scenarios |
| Backtesting | Test strategy over long history |
| Optimization | Evaluate objective function many times |
| Bootstrap inference | Resample and calculate statistics |

---

## Exercises

### Exercise 1: European Put Option

Modify the Monte Carlo option pricing code to price a European put option.

In [ ]:
# Your code here
def monte_carlo_put_batch(args):
    """Price a European put option using Monte Carlo."""
    pass  # Implement me!

### Exercise 2: Optimal Chunk Size

Experiment with different chunk sizes for the small_task example. Find the optimal chunk size for your machine.

In [ ]:
# Your code here
# Try chunk sizes: 10, 50, 100, 250, 500, 1000
# Measure execution time for each

### Exercise 3: Asian Option Pricing

Price an Asian call option (payoff based on average price over the life of the option) using parallel Monte Carlo.

In [ ]:
# Your code here
# Hint: You'll need to simulate the full price path, not just the terminal price